# Data Modeling for NAACC + Capacity Modeling

Working with Drainit models (`models/__init__.py`); testing ETL processes into models for NAACC table-based capacity calculations.

In [1]:
import json
from pathlib import Path
from typing import List
from collections import OrderedDict
from dataclasses import field, asdict, replace
import math

import petl as etl
import pint
import marshmallow
from marshmallow import Schema, fields, EXCLUDE, pre_load, ValidationError
import marshmallow_dataclass
from marshmallow_dataclass import dataclass, class_schema

In [2]:
units = pint.UnitRegistry()

In [3]:
# -------------------------------------
# RAINFALL

@dataclass
class RainfallRaster:
    """store a reference to a NOAA Rainfall raster
    """

    path: str = None
    freq: int = None
    ext: str = None

RainfallRasterSchema = class_schema(RainfallRaster)


@dataclass
class RainfallRasterConfig:
    """store rainfall download metadata with methods for portability
    """

    root: str = None
    rasters: List[RainfallRaster] = field(default_factory=[])

RainfallRasterConfigSchema = class_schema(RainfallRasterConfig)

In [4]:
Runoff = str
Overflow = str

In [5]:
def _culvert_capacity(
    culvert_area_sqm, 
    head_over_invert, 
    culvert_depth_m, 
    slope_rr, 
    coefficient_slope=-0.5, 
    coefficient_y=-0.04,
    coefficient_c=0.7, 
    si_conv_factor=1.811
    ):
    
    # Calculate and return the capacity for the culvert and store with the rest of the data for that culvert.
    try:
        return (culvert_area_sqm * math.sqrt(culvert_depth_m * ((head_over_invert / culvert_depth_m) - coefficient_y - coefficient_slope * slope_rr) / coefficient_c)) / si_conv_factor
    except:
        return -9999

In [6]:
@dataclass
class Capacity:
    """Model for culvert capacity. Includes paramters both crosswalked and 
    derived from the NAACC data required for the culvert capacity calculation.
    """

    # ----------------------------
    # cross-walked attributes:

    culv_mat: str = None
    in_type: str = None
    in_shape: str = None
    in_a: float = None
    in_b: float = None
    hw: float = None
    slope: float = None
    length: float = None
    out_shape: str = None
    out_a: float = None
    out_b: float = None
    crossing_type: str = None
        
    #flags: int = 1

    # ----------------------------
    # derived attributes

    # culvert area (square meters)
    culvert_area_sqm: float = None
    # culvert depth (meters)
    culvert_depth_m: float = None
    # coefficients based on shape and material from FHWA engineering pub HIF12026, appendix A
    coefficient_c: float = 0.04
    coefficient_y: float = 0.7
    # slope coefficient from FHWA engineering pub HIF12026, appendix A
    coefficient_slope: float = -0.5
    # slope as rise/run
    slope_rr: float = None
    #  head over invert by adding dist from road to top of culvert to D 
    head_over_invert: float = None

    # comment field
    comments: list = field(default_factory=list)
    
    # include flag
    include: bool = True

    # culvert capacity, in cubic meters / second (m^3/s)
    culvert_capacity: float = None

    def calculate(self, si_conv_factor=1.811):
        
        self.culvert_capacity = _culvert_capacity(
            culvert_area_sqm=self.culvert_area_sqm, 
            head_over_invert=self.head_over_invert, 
            culvert_depth_m=self.culvert_depth_m, 
            slope_rr=self.slope_rr, 
            coefficient_slope=self.coefficient_slope, 
            coefficient_y=self.coefficient_y,
            coefficient_c=self.coefficient_c, 
            si_conv_factor=si_conv_factor
        )
        return self.culvert_capacity        
        
    class Meta:
        unknown = EXCLUDE        
        
capacity_numeric_fields = {k: v.type for k, v in Capacity.__dataclass_fields__.items() if v.type in [int, float]}

# NaaccCulvertFields = NaaccCulvert.__dataclass_fields__
CapacitySchema = class_schema(Capacity)

In [7]:
# -------------------------------------
# ANALYSIS RESULTS

@dataclass
class Analytics:
    """analysis results from the calculators
    """

    runoff: Runoff = None

    # requires naacc_capacity:
    overflow: Overflow = None 


@dataclass
class Frequency:
    """storm frequency interval with rainfall values and rainfall-dependent 
    analytical results from the calculators
    """    

    year: int = None
    rainfall: float = None
    analytics: Analytics = None

FrequencySchema = class_schema(Frequency)

In [8]:
# -------------------------------------
# LOCATION TYPES

def req_field(): 
    return field(metadata=dict(required=True))

def cast_to_numeric_fields(data, dataclass_model, **kwargs):
    """when loading or validating, attempt to cast numbers from strings based on the model field types."""
    numeric_fields = {k: v.type for k, v in dataclass_model.__dataclass_fields__.items() if v.type in [int, float]}
    for fld, typ in numeric_fields.items():
        if fld in data.keys():
            if not isinstance(data[fld], typ) and data[fld] is not None:
                #print(data[fld], type(data[fld]), isinstance(data[fld], typ), type(data[fld]) is not None)
                try:
                    data[fld] = typ(data[fld])
                except ValueError as e:
                    #print(e, fld, data[fld])
                    pass
    return data

 
@dataclass
class NaaccCulvert:
    """NAACC model for a single culvert. Use primarily for validating and 
    type-casting incoming NAACC CSVs.
    
    NOTE: this is only a subset of available NAACC fields
    """

    Naacc_Culvert_Id: str = req_field() # 'field_short': 'NAACC_ID'
    Survey_Id: str = req_field() # 'field_short': 'Survey_ID'

    GIS_Latitude: float = req_field() # 'field_short': 'Lat'
    GIS_Longitude: float = req_field() # 'field_short': 'Long'        
        
    Number_Of_Culverts: int = req_field() # 'field_short': 'Flags'

    Material: str = req_field() # 'field_short': 'Culv_Mat'
    Inlet_Type: str = req_field() # 'field_short': 'In_Type'
    Inlet_Structure_Type: str = req_field() # 'field_short': 'In_Shape'

    Inlet_Width: float = req_field() # 'field_short': 'In_A'
    Inlet_Height: float = req_field() # 'field_short': 'In_B'
    Road_Fill_Height: float = req_field() # 'field_short': 'HW'
    Slope_Percent: float = req_field() # 'field_short': 'Slope'
    Crossing_Structure_Length: float = req_field() # 'field_short': 'Length'
    Outlet_Structure_Type: str = req_field() # 'field_short': 'Out_Shape'
    Outlet_Width: float = req_field() # 'field_short': 'Out_A'
    Outlet_Height: float = req_field() # 'field_short': 'Out_B'
    Crossing_Type: str = req_field() # 'field_short': 'Crossing_Type'
    
    Road: str = None # 'field_short': 'Rd_Name'
    Crossing_Comment: str = None # 'field_short': 'Comments'


    @pre_load
    def cast_numeric_fields(self, data, **kwargs):
        """when loading or validating, attempt to cast numbers from strings before checking."""
        return cast_to_numeric_fields(data, NaaccCulvert, **kwargs)
    
    class Meta:
        unknown = EXCLUDE    

NaaccCulvertSchema = class_schema(NaaccCulvert)


@dataclass
class NaaccCrossing:
    """a model for representing multiple Culverts NaaccPoints
    """

    crossing_id: str
    culverts: List[NaaccCulvert]

NaaccCrossingSchema = class_schema(NaaccCrossing)

In [9]:
@dataclass
class Shed:
    """Characteristics of a single point's contributing area
    """
    # unique id field, derived from the outlet point; the value from the
    # "pour_point_field". For NAACC-based culvert modeling, this is the
    # NAACC Naacc_Culvert_Id field
    uid: str = None
    
    # a group id field. non-unique ID field that indicates groups of related
    # outlets. Used primarily for NAACC-based culvert modeling, this is the
    # NAACC Survey_Id field
    group_id: str = None

    # characteristics used for calculating peak flow
    area_sqkm: float = None# <area of inlet's catchment in square km>
    avg_slope_pct: float = None # <average slope of DEM in catchment>
    avg_cn: float = None # <average curve number in the catchment>
    max_fl: float = None # <maximum flow length in the catchment>
    rainfall: float = None # average rainfall in the catchment

    # geometries
    inlet_geom: str = None
    shed_geom: str = None
    
    # for recording the location of intermediate geospatial output files
    shed_polygon_filepath: str = None
    shed_raster_filepath: str = None

ShedSchema = class_schema(Shed)


@dataclass
class Point:
    """Basic model for points used as source delineations for peak-flow-calcs;
    minimal attributes required.
    """
    # unique id field, derived from the outlet point; the value from the
    # "pour_point_field". For NAACC-based culvert modeling, this is the
    # NAACC Naacc_Culvert_Id field
    uid: str

    lat: float
    lng: float

    # a group id field. non-unique ID field that indicates groups of related
    # outlets. Used primarily for NAACC-based culvert modeling, this is the
    # NAACC Survey_Id field
    group_id: str = None

    # optionally extend with NAACC attributes
    naacc: NaaccCulvert = None
    capacity: Capacity = None

    # optionally extend with the Shed and its characteristics
    shed: Shed = None

    # Analytical results associated with the Point by storm frequency
    analytics: List[Frequency] = field(default_factory=list)
        
    include: bool = True
    validation_errors: list = field(default_factory=list)
    notes: str = ""

    # place to store the raw input
    raw: dict = None        

PointSchema = class_schema(Point)

# ------------------------------------------------------------------------------
# WORKFLOW MODELS

@dataclass
class WorkflowConfig:
    """Store all parameters required for any of our model runs.
    """

    # directories
    work_dir: str = None

    # -----------------------------
    # input points (culverts or catch-basins)

    points_filepath: str = None
    points_features: dict = None
    points_id_fieldname: str = None
    is_naacc: bool = False
    
    # -----------------------------
    # input landscape rasters

    # optional for peak-flow-calc
    raster_dem_filepath: str = None
    raster_watershed_filepath: str = None

    # required for peak-flow-calc (can be derived)
    raster_flowdir_filepath: str = None
    raster_slope_filepath: str = None
    raster_curvenumber_filepath: str = None

    # --------------------------
    # input rainfall

    precip_src_config_filepath: str = None
    precip_noaa_csv_filepath: str = None

    rainfall_rasters: List[RainfallRaster] = field(default_factory=list)

    # --------------------------
    # analysis parameters
    
    area_conv_factor: float = 0.00000009290304
    leng_conv_factor: float = 1

    basins_simplify: bool = False
    basins_in_series: bool = True

    # --------------------------
    # file output parameters
    output_points_filepath: str = None
    output_basins_filepath: str = None

    # --------------------------
    # intermediate and internal data

    all_basins_raster: str = None
    all_basins_vector: str = None

    points: List[Point] = field(default_factory=list)
    basins: List[Shed] = field(default_factory=list)


WorkflowConfigSchema = class_schema(WorkflowConfig)

## helper functions:

In [10]:
def validate_petl_record_w_schema(row, schema):
    r = {i[0]: i[1] for i in zip(row.flds, row)}
    #errors = schema.load(r, unknown=marshmallow.EXCLUDE)
    errors = schema.validate(r)
    #errors = {k: v for k,v in errors.items() if v[0] != 'Unknown field.'}
    if errors:
        return errors
    return None

def convert_value_via_xwalk(
    k, crosswalk, preserve_non_matches=True, no_match_value=None
    ):
    if k in crosswalk.keys():
        return crosswalk[k]
    else:
        if preserve_non_matches:
            return k
        else:
            return no_match_value

## Constants and Lookups

In [11]:
NAACC_INLET_SHAPE_CROSSWALK = {
    'Round Culvert': 'Round',
    'Pipe Arch/Elliptical Culvert': 'Elliptical',
    'Box Culvert': 'Box',
    'Box/Bridge with Abutments': 'Box',
    'Bridge with Abutments and Side Slopes': 'Box',
    'Open Bottom Arch Bridge/Culvert': 'Arch'
}

NAACC_INLET_TYPE_CROSSWALK = {
    "Headwall and Wingwalls": "Wingwall and Headwall",
    "Wingwalls": "Wingwall",
    "None": "Projecting"
}

# crosswalk NaaccCulvert fields with Capacity fields
NAACC_HEADER_XWALK = [
#     {'field_idx': 0, 'field_name': 'Survey_Id', 'field_short': 'group_id' },
#     {'field_idx': 35, 'field_name': 'Naacc_Culvert_Id', 'field_short': 'uid'},
#     {'field_idx': 20, 'field_name': 'GIS_Latitude', 'field_short': 'lat', 'field_type': float},
#     {'field_idx': 19, 'field_name': 'GIS_Longitude', 'field_short': 'lng', 'field_type': float},
    {'field_idx': 26, 'field_name': 'Road', 'field_short': 'rd_name'},
    {'field_idx': 49, 'field_name': 'Material', 'field_short': 'culv_mat'},
    {'field_idx': 22, 'field_name': 'Inlet_Type', 'field_short': 'in_type'},
    {'field_idx': 44, 'field_name': 'Inlet_Structure_Type', 'field_short': 'in_shape'},
    {'field_idx': 47, 'field_name': 'Inlet_Width', 'field_short': 'in_a', 'field_type': float},
    {'field_idx': 43, 'field_name': 'Inlet_Height', 'field_short': 'in_b', 'field_type': float},
    {'field_idx': 27, 'field_name': 'Road_Fill_Height', 'field_short': 'hw', 'field_type': float},
    {'field_idx': 61, 'field_name': 'Slope_Percent', 'field_short': 'slope', 'field_type': float},
    {'field_idx': 39, 'field_name': 'Crossing_Structure_Length', 'field_short': 'length', 'field_type': float},
    {'field_idx': 55, 'field_name': 'Outlet_Structure_Type', 'field_short': 'out_shape'},
    {'field_idx': 58, 'field_name': 'Outlet_Width', 'field_short': 'out_a', 'field_type': float},
    {'field_idx': 54, 'field_name': 'Outlet_Height', 'field_short': 'out_b', 'field_type': float},
    {'field_idx': 11, 'field_name': 'Crossing_Type', 'field_short': 'crossing_type'},
    {'field_idx': 8, 'field_name': 'Crossing_Comment', 'field_short': 'comments'},
#     {'field_idx': 24, 'field_name': 'Number_Of_Culverts', 'field_short': 'flags', 'field_type': int}
]

NAACC_HEADER_LOOKUP = {i['field_name']: i['field_short'] for i in NAACC_HEADER_XWALK}
# NAACC_HEADER_LOOKUP_FOR_COPY = {i['field_short']: i['field_name'] for i in NAACC_HEADER_XWALK}

NAACC_TYPECASTS = {
    i['field_short']: i['field_type'] 
    for i in 
    NAACC_HEADER_XWALK
    if 'field_type' in i.keys()
}

NAACC_TYPECASTS_FULLNAME = {
    i['field_name']: i['field_type'] 
    for i in 
    NAACC_HEADER_XWALK
    if 'field_type' in i.keys()
}

In [12]:
lookup_naac_inlet_shape=NAACC_INLET_SHAPE_CROSSWALK
lookup_naac_inlet_type=NAACC_INLET_TYPE_CROSSWALK

## NAACC utilities

In [13]:
def _copy_naac_to_capacity(row):
    
    r = OrderedDict({i[0]: i[1] for i in zip(row.flds, row)})
    
    # if r['validation_errors'] is not None:
    #   return tuple(r.values())
    
    for n_field, cap_field in NAACC_HEADER_LOOKUP.items():
        if cap_field in capacity_numeric_fields.keys():
            typ = capacity_numeric_fields[cap_field]
            try:
                r[cap_field] = typ(r[n_field])
            except:
                r[cap_field] = r[n_field]
        
    return tuple(r.values())

In [14]:
def _naacc_exclude_tests(row):
    """helper function for the NAACC ETL process, used within the context of a petl.rowmap function;
    writes to a boolean include column and notes based on any number of conditions conditions;
    """

    # convert PETL Record object to an ordered dictionary
    r = OrderedDict({i[0]: i[1] for i in zip(row.flds, row)})
    
    #if r['validation_errors'] is not None:
    #   return tuple(r.values())
    
    try:
        
        exclusion_comments = []

        # wrong bridge type
        if all([
            r["crossing_type"] == "Bridge",
            r["in_shape"]
            not in ["Box/Bridge with Abutments", "Open Bottom Arch Bridge/Culvert"],
        ]):
            r["exclude"] = True
            exclusion_comments.append("Wrong bridge type")

        # wrong bridge width
        if all([
            r["crossing_type"] == "Bridge", 
            r["in_a"] is not None and r["in_a"] >= 20
        ]):
            r["exclude"] = True
            exclusion_comments.append("Bridge wider than 20 ft")

        # bad geometry
        if not all([
            isinstance(r["in_a"], float),
            isinstance(r["in_b"], float),
            isinstance(r["hw"], float),
            isinstance(r["length"], float)
        ]):
            r["include"] = False
            exclusion_comments.append("Required culvert geometry is missing")
        elif any([
            r["in_a"] < 0,
            r["in_b"] < 0,
            r["hw"] < 0,
            r["length"] < 0
        ]):
            r["include"] = False
            exclusion_comments.append("Required culvert geometry is negative.")
        else:
            pass
        
        if exclusion_comments:
            if r['validation_errors'] is not None:
                r['validation_errors']['Capacity'] = exclusion_comments
            else:
                r['validation_errors'] = {'Capacity': exclusion_comments}

        # return as a tuple of values for the row
        return tuple(r.values())
    
    except TypeError as e:
        print('_naacc_exclude_tests', e, r['Survey_Id'])
        #print(r)
        return tuple(r.values())

In [15]:
def _derive_capacity_parameters(row):
    """transform or derive values for use in calculating culvert capacity

    NOTE: This function is designed to be used per-row within a petl.rowmap 
    function call
    
    TODO: replace the nested if/elif/else business logic here with a multi-column lookup table
    that can be loaded as a config file.

    :param row: a single NAACC table row, where the table is from core.logic.data_io.extract_naacc_table
    :type row: petl.Record
    :return: the row, w/ transformed or derived values
    :rtype: tuple
    """

    # convert the incoming PETL.Record object to a dictionary
    row = OrderedDict({i[0]: i[1] for i in zip(row.flds, row)})
    
    if row['validation_errors'] is not None:
      return tuple(row.values())

    try:

        # -----------------------------------------------------
        # constants 

        # pi. Note that source script used a precision 5 float instead of 
        # Python's available math.pi constant, the latter likely being more precise
        pi = 3.14159 #math.pi

        # -----------------------------------------------------
        # variables to be calculated

        # culvert area ( square meters; default is for round pipe)
        culvert_area_sqm = ((row["in_a"] / 2) ** 2) * pi
        # culvert depth (meters, default is for round pipe)
        culvert_depth_m = row["in_a"]
        # coefficients based on shape and material from FHWA engineering pub HIF12026, appendix A
        coefficient_c = 0.04
        coefficient_y = 0.7
        # slope coefficient from FHWA engineering pub HIF12026, appendix A
        coefficient_slope = -0.5
        # slope as rise/run
        slope_rr = 0
        #  head over invert by adding dist from road to top of culvert to D 
        head_over_invert = 0
        
        exclusion_comments = []

        # -----------------------------------------------------
        # imperial to metric conversions

        row["length"] = (row["length"] * units.foot).to(units.meter).magnitude
        row["in_a"] = (row["in_a"] * units.foot).to(units.meter).magnitude
        row["hw"] = (row["hw"] * units.foot).to(units.meter).magnitude

        # if culvert is not round, need B (height), so convert from feet to meters
        if row["in_shape"] != "Round":
            row["in_b"] = (row["in_b"] * units.foot).to(units.meter).magnitude

        # -----------------------------------------------------
        # culvert slope as rise/run
        slope_rr = row["slope"] / 100

        # -----------------------------------------------------
        # calculate culvert area and depth based on culvert shape

         # if culvert is round, depth is diameter
        if row["in_shape"] == "Round":
            culvert_area_sqm = ((row["in_a"] / 2) ** 2) * pi  # Area in m^2, thus diameter in m
            culvert_depth_m = row["in_a"]
        # if culvert is eliptical, depth is B 
        elif row["in_shape"] in ["Elliptical", "Pipe Arch"]:
            culvert_area_sqm = (row["in_a"] / 2) * (row["in_b"] / 2) * pi
            culvert_depth_m = row["in_b"]
        # if culvert is a box, depth is B
        elif row["in_shape"] == "Box":
            culvert_area_sqm = (row["in_a"]) * (row["in_b"])
            culvert_depth_m = row["in_b"]
        # if culvert is an arch, depth is B
        elif row["in_shape"] == "Arch":
            culvert_area_sqm = ((row["in_a"] / 2) * (row["in_b"] / 2) * pi) / 2
            culvert_depth_m = row["in_b"]
#         else:
#             exclusion_comments.append('in_shape has unknown value: {0}'.format(row["in_shape"]))

        # Calculate head over invert by adding dist from road to top of culvert to D
        # H = row['HW'] / 3.2808 + D
        head_over_invert = row["hw"] + culvert_depth_m  # TODO: Check if OK that conversion is done above

        # assign ks (slope coefficient from FHWA engineering pub HIF12026, appendix A)
        if row["in_type"] == "Mitered to Slope":
            coefficient_slope = 0.7
        else:
            coefficient_slope = -0.5

        # assign c and y values (coefficients based on shape and material from FHWA engineering pub HIF12026, appendix A)
        # no c and y value provide for inlet_type == "other".  Will take on the filler values
        if row["in_shape"] == "Arch":
            if row["culv_mat"] in ["Concrete", "Stone"]:
                if row["in_type"] in ["Headwall", "Projecting"]:
                    coefficient_c = 0.041
                    coefficient_y = 0.570
                elif row["in_type"] == "Mitered to Slope":
                    coefficient_c = 0.040
                    coefficient_y = 0.48
                elif row["in_type"] == "Wingwall":
                    coefficient_c = 0.040
                    coefficient_y = 0.620
                elif row["in_type"] == "Wingwall and Headwall":
                    coefficient_c = 0.040
                    coefficient_y = 0.620
#                 else:
#                     exclusion_comments.append(
#                         'in_shape+culv_mat+in_type has unhandled combination of values: [{0}]'\
#                         .format(" + ".join([row["in_shape"], row["culv_mat"], row["in_type"]]))
#                     )
                    
            elif (row["culv_mat"] in ["Plastic", "Metal"]):  
                # inlet_type to row['Culv_Mat'] for plastic - sharon
                if row["in_type"] == "Mitered to Slope":
                    coefficient_c = 0.0540
                    coefficient_y = 0.5
                elif row["in_type"] == "Projecting":
                    coefficient_c = 0.065
                    coefficient_y = 0.12
                elif any(
                    [
                        row["in_type"] == "Headwall",
                        row["in_type"] == "Wingwall and Headwall",
                        row["in_type"] == "Wingwall",
                    ]
                ):
                    coefficient_c = 0.0431
                    coefficient_y = 0.610
#                 else:
#                     exclusion_comments.append(
#                         'in_shape+culv_mat+in_type has unhandled combination of values: [{0}]'\
#                         .format(" + ".join([row["in_shape"], row["culv_mat"], row["in_type"]]))
#                     )                    
            elif row["culv_mat"] == "Combination":
                coefficient_c = 0.045 # Changed March 2019 from c = 1.0   #filler values -sharon
                coefficient_y = 0.5  # Y = 1.0    # filler values - sharon
                row["comments"].append("Filler c & Y values.")
#             else:
#                 exclusion_comments.append(
#                     'in_shape+culv_mat has unhandled combination of values: [{0}]'\
#                     .format(" + ".join([row["in_shape"], row["culv_mat"]]))
#                 )
                

        elif row["in_shape"] == "Box":
            if row["culv_mat"] in ["Concrete", "Stone"]:
                coefficient_c = 0.0378
                coefficient_y = 0.870
            elif row["culv_mat"] in ["Plastic", "Metal"]:
                if row["in_type"] == "Headwall":
                    coefficient_c = 0.0379
                    coefficient_y = 0.690  # put in else statement in case other inlet types exist-Sharon
                elif row["in_type"] == "Wingwall":  ## Jo put this in but needs to check...
                    coefficient_c = 0.040
                    coefficient_y = 0.620
                    row["comments"].append("Filler c & Y values.")
                else:
                    coefficient_c = 0.04  # c = 1.0
                    coefficient_y = 0.65  # Y = 1.0 #filler numbers -Sharon
                    row["comments"].append("Filler c & Y values.")
            elif row["culv_mat"] == "Wood":
                coefficient_c = 0.038
                coefficient_y = 0.87
            elif row["culv_mat"] == "Combination":
                coefficient_c = 0.038
                coefficient_y = 0.7  # filler values -Sharon
                row["comments"].append("Filler c & Y values.")
#             else:
#                 exclusion_comments.append(
#                     'in_shape+culv_mat has unhandled combination of values: [{0}]'\
#                     .format(" + ".join([row["in_shape"], row["culv_mat"]]))
#                 )

        elif row["in_shape"] in ["Elliptical", "Pipe Arch"]:
            if row["culv_mat"] in ["Concrete", "Stone"]:
                coefficient_c = 0.048
                coefficient_y = 0.80
            elif row["culv_mat"] in ["Plastic", "Metal"]:
                if row["in_type"] == "Projecting":
                    coefficient_c = 0.060
                    coefficient_y = 0.75
                else:
                    coefficient_c = 0.048
                    coefficient_y = 0.80
            elif row["culv_mat"] == "Combination":
                coefficient_c = 0.05  # c = 1.0
                coefficient_y = 0.8  # Y = 1.0  #filler -Sharon
                row["comments"].append("Filler c & Y values.")
#             else:
#                 exclusion_comments.append(
#                     'in_shape+culv_mat has unhandled combination of values: [{0}]'\
#                     .format(" + ".join([row["in_shape"], row["culv_mat"]]))
#                 )

        elif row["in_shape"] == "Round":
            if row["culv_mat"] in ["Concrete", "Stone"]:
                if row["in_type"] == "Projecting":
                    coefficient_c = 0.032
                    coefficient_y = 0.69
                else:
                    coefficient_c = 0.029
                    coefficient_y = 0.74
            elif row["culv_mat"] in ["Plastic", "Metal"]:
                if row["in_type"] == "Projecting":
                    coefficient_c = 0.055
                    coefficient_y = 0.54
                elif row["in_type"] == "Mitered to Slope":
                    coefficient_c = 0.046
                    coefficient_y = 0.75
                else:
                    coefficient_c = 0.038
                    coefficient_y = 0.69
            elif row["culv_mat"] == "Combination":
                coefficient_c = 0.04  # c = 1.0
                coefficient_y = 0.65  # Y = 1.0 #filler-Sharon
                row["comments"].append("Filler c & Y values.")
#             else:
#                 exclusion_comments.append(
#                     'in_shape+culv_mat has unhandled combination of values: [{0}]'\
#                     .format(" + ".join([row["in_shape"], row["culv_mat"]]))
#                 )
                
#         else:
#             exclusion_comments.append(
#                 'in_shape has unhandled value: [{0}]'\
#                 .format(row["in_shape"])
#             )

        # store computed values in the row dictionary
        row['culvert_area_sqm'] = culvert_area_sqm
        row['culvert_depth_m'] = culvert_depth_m
        row['coefficient_c'] = coefficient_c
        row['coefficient_y'] = coefficient_y
        row['coefficient_slope'] = coefficient_slope
        row['head_over_invert'] = head_over_invert
        row['slope_rr'] = slope_rr
        
        if exclusion_comments:
            if row['validation_errors'] is not None:
                row['validation_errors']['Capacity_Params'] = exclusion_comments
            else:
                row['validation_errors'] = {'Capacity_Params': exclusion_comments}

        # return the row in the format expected by PETL.rowmap
        return tuple(row.values())
    
    except TypeError as e:
        print(e, row['Survey_Id'])
        print(row)
        print(e.with_traceback())
        return tuple(row.values())

## ETL CSV to PETL table

Transform missing values to None; validate each record in new column using the NAACC model schema; add additional meta columns.

In [16]:
raw_table = etl.fromcsv(r"D:\Dropbox (CivicMapper)\Projects\202004-02 Cornell Modeling\3 - Production\tool outputs\c19 baseline\C19\C19.csv")

In [17]:
naacc_culvert_schema = NaaccCulvertSchema()

validated_table = etl\
    .replaceall(raw_table, "", None)\
    .addfield(
        'validation_errors', 
        lambda rec: validate_petl_record_w_schema(rec, naacc_culvert_schema)
    )

In [18]:
bad = etl.selectnotnone(validated_table, 'validation_errors')
etl.nrows(bad)

78

## Derive params used for capacity & overflow calculations

Add fields from the Capacity model to the table and crosswalk to generic fields and values.

* add capacity fields
* copy values from naacc fields / convert values using lookups
* set include/exclude based on capacity field values

In [19]:
extended_table = etl\
    .addfields(
        validated_table, 
        [(k, v.default) for k,v in Capacity.__dataclass_fields__.items()]
    )

In [20]:
extended_table_header = list(etl.header(extended_table))

In [21]:
hydrated_table = etl\
    .rowmap(
        extended_table,
        _copy_naac_to_capacity, 
        header=extended_table_header, 
        failonerror=True
    )\
    .convert(
        "in_shape",
        lambda v, r: convert_value_via_xwalk(r['Inlet_Structure_Type'], lookup_naac_inlet_shape),
        failonerror=True,
        pass_row=True
    )\
    .convert(
        "in_type",
        lambda v, r: convert_value_via_xwalk(r['Inlet_Type'], lookup_naac_inlet_type),
        failonerror=True,
        pass_row=True
    )\
    .convert("comments",lambda v: [])\
    .rowmap(
        _naacc_exclude_tests, 
        header=extended_table_header, 
        failonerror=True
    )\
    .rowmap(
        _derive_capacity_parameters,
        header=extended_table_header, 
        failonerror=True
    )

#hydrated_table

In [22]:
bad = etl.selectnotnone(hydrated_table, 'validation_errors')
etl.nrows(bad)

97

In [23]:
#[i['validation_errors'] for i in etl.dicts(bad) if 'Capacity' not in i['validation_errors'].keys()]

In [24]:
#[i['validation_errors']['Capacity'] for i in etl.dicts(bad) if 'Capacity' in i['validation_errors'].keys()]

In [25]:
etl.selecteq(hydrated_table, 'Naacc_Culvert_Id', '64678')

Survey_Id,Crossing_Code,Alignment,Aqua_Pass_Score,AOP,Approved,Bankfull_Width,Bankfull_Width_Confidence,Coordinator,Crossing_Comment,Crossing_Condition,Crossing_Span,Crossing_Type,Data_Checked_Coordinator,Database_Entry_By,Date_Data_Checked,Date_First_Entered,Date_Last_Updated,Date_Observed,Evaluation,Flag_Name,Flow_Condition,GIS_Latitude,GIS_Longitude,GPS_X_Coordinate,GPS_Y_Coordinate,GPS_Distance,Inlet_Type,Lccx,Lccy,Lccx_Moved,Lccy_Moved,Local_Id,Location_Description,Maine_Private,No_Crossing,Number_Of_Culverts,Observer,Road,Road_Fill_Height,Road_Type,Scour_Pool,State,Stream_Name,Terrestrial_Passage_Score,Tidal_Site,Town,Naacc_Culvert_Id,Armoring,Barrier_Name,Barrier_Severity,Crossing_Structure_Length,Culvert_Condition_Assess_Id,Dry_Passage,Inlet_Abutment_Height,Inlet_Grade,Inlet_Height,Inlet_Openness,Inlet_Structure_Type,Inlet_Substrate_Water_Width,Inlet_Type1,Inlet_Water_Depth,Inlet_Width,Internal_Structure,Internal_Structure_Comment,Material,NHD_HUC8_Watershed,Outlet_Drop_To_Stream_Bottom,Outlet_Drop_To_Water_Surface,Outlet_Grade,Outlet_Height,Outlet_Openness,Outlet_Structure_Type,Outlet_Substrate_Water_Width,Outlet_Water_Depth,Outlet_Width,Passage_Height,Slope_Confidence,Slope_Percent,Structure_Comment,Structure_Substrate_Matches_Stream,Substrate_Continuous,Substrate_Type,Water_Depth_Matches_Stream,Water_Velocity,County,validation_errors,culv_mat,in_type,in_shape,in_a,in_b,hw,slope,length,out_shape,out_a,out_b,crossing_type,culvert_area_sqm,culvert_depth_m,coefficient_c,coefficient_y,coefficient_slope,slope_rr,head_over_invert,comments,include,culvert_capacity
66756,xy4210100273589809,Flow-Aligned,0.883287596,Reduced AOP,true,9,High,"Jastremski, Michael",None,OK,Moderate,Culvert,"Jastremski, Michael",1774,2019-01-23 09:40:41.53,2019-01-23 09:34:10.417,2019-01-23 09:34:11.01,2019-01-18,Insignificant barrier,No data,Typical low-flow,42.101002,-73.589809,-73.589812,42.100996,0.7,Headwall and Wingwalls,1823923.599,2338265.866,1823923.599,2338265.866,None,Green Tile drain sign,false,false,1,"Larson, Lindsay",County Route 7,2.5,Paved,Small,NY,Unnamed,None,No,Copake,64678,None,None,None,43,0,No,-1,At Stream Grade,3,0.351,Box Culvert,5.8,Headwall and Wingwalls,0.4,5.8,None,None,Concrete,Middle Hudson,0,0,At Stream Grade,3.4,0.401,Box Culvert,4.9,0.21,5.4,-1,Low,0.1,No data,Contrasting,100%,Sand,Yes,Yes,Columbia,None,None,Wingwall and Headwall,Box,1.7678399999999996,0.9143999999999999,0.7619999999999999,0.1,13.106399999999999,None,5.4,3.4,None,1.6165128959999995,0.9143999999999999,0.04,0.7,-0.5,0.001,1.6763999999999997,[],True,None


## Load into our Point and nested NAACC dataclasses

In [26]:
capacity_schema = CapacitySchema()

In [27]:
points = []
for idx, r in enumerate(list(etl.dicts(hydrated_table))):
    
    kwargs = dict(
        uid=r["Naacc_Culvert_Id"],
        group_id=r["Survey_Id"],
        lat=float(r["GIS_Latitude"]),
        lng=float(r["GIS_Longitude"]),
        include = r['include'],
        raw=r
    )
    
    if r['validation_errors']:
        kwargs['validation_errors'] = {'naacc': r['validation_errors']}
    
    try:
        
        naacc = naacc_culvert_schema.load(data=r)
        capacity = capacity_schema.load(data=r)
        capacity.calculate()
        
        kwargs['naacc'] = naacc
        kwargs['capacity'] = capacity
        
    except ValidationError as e:
        #print(idx, r["Naacc_Culvert_Id"], e, "\n")
        pass
    
    p = Point(**kwargs)
    points.append(p)

In [28]:
points[10].capacity

Capacity(culv_mat=None, in_type='Wingwall and Headwall', in_shape='Box', in_a=1.7678399999999996, in_b=0.9143999999999999, hw=0.7619999999999999, slope=0.1, length=13.106399999999999, out_shape=None, out_a=5.4, out_b=3.4, crossing_type=None, culvert_area_sqm=1.6165128959999995, culvert_depth_m=0.9143999999999999, coefficient_c=0.04, coefficient_y=0.7, coefficient_slope=-0.5, slope_rr=0.001, head_over_invert=1.6763999999999997, comments=[], include=True, culvert_capacity=4.54436751455028)

### Then we save out this to CSV:

In [43]:
hydrated_table

Survey_Id,Crossing_Code,Alignment,Aqua_Pass_Score,AOP,Approved,Bankfull_Width,Bankfull_Width_Confidence,Coordinator,Crossing_Comment,Crossing_Condition,Crossing_Span,Crossing_Type,Data_Checked_Coordinator,Database_Entry_By,Date_Data_Checked,Date_First_Entered,Date_Last_Updated,Date_Observed,Evaluation,Flag_Name,Flow_Condition,GIS_Latitude,GIS_Longitude,GPS_X_Coordinate,GPS_Y_Coordinate,GPS_Distance,Inlet_Type,Lccx,Lccy,Lccx_Moved,Lccy_Moved,Local_Id,Location_Description,Maine_Private,No_Crossing,Number_Of_Culverts,Observer,Road,Road_Fill_Height,Road_Type,Scour_Pool,State,Stream_Name,Terrestrial_Passage_Score,Tidal_Site,Town,Naacc_Culvert_Id,Armoring,Barrier_Name,Barrier_Severity,Crossing_Structure_Length,Culvert_Condition_Assess_Id,Dry_Passage,Inlet_Abutment_Height,Inlet_Grade,Inlet_Height,Inlet_Openness,Inlet_Structure_Type,Inlet_Substrate_Water_Width,Inlet_Type1,Inlet_Water_Depth,Inlet_Width,Internal_Structure,Internal_Structure_Comment,Material,NHD_HUC8_Watershed,Outlet_Drop_To_Stream_Bottom,Outlet_Drop_To_Water_Surface,Outlet_Grade,Outlet_Height,Outlet_Openness,Outlet_Structure_Type,Outlet_Substrate_Water_Width,Outlet_Water_Depth,Outlet_Width,Passage_Height,Slope_Confidence,Slope_Percent,Structure_Comment,Structure_Substrate_Matches_Stream,Substrate_Continuous,Substrate_Type,Water_Depth_Matches_Stream,Water_Velocity,County,validation_errors,culv_mat,in_type,in_shape,in_a,in_b,hw,slope,length,out_shape,out_a,out_b,crossing_type,culvert_area_sqm,culvert_depth_m,coefficient_c,coefficient_y,coefficient_slope,slope_rr,head_over_invert,comments,include,culvert_capacity
66697,xy4216342773600046,No data,-1,no score - missing data,true,-1,No data,"Jastremski, Michael","Long hike in, off gated driveway",No data,No data,Inaccessible,"Jastremski, Michael",1774,2019-01-10 10:06:43.467,2019-01-10 10:01:41.81,2019-01-10 10:01:41.933,2018-08-14,no score - missing data,No data,No data,42.163427,-73.600046,-73.600091,42.163404,4.5,None,1821475.438,2344858.227,1821475.438,2344858.227,None,"Hike in, possible dirt path off driveway 115 Copake Lake Road",false,false,-1,"Larson, Lindsay",None,-1,Trail,No data,NY,Unnamed,None,No data,Copake,None,None,No culvert,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Middle Hudson,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Columbia,"{'Inlet_Type': ['Field may not be null.'], 'Inlet_Height': ['Field may not be null.'], 'Material': ['Field may not be null.'], 'Naacc_Culvert_Id': ['Field may not be null.'], 'Inlet_Structure_Type': ['Field may not be null.'], 'Outlet_Structure_Type': ['Field may not be null.'], 'Slope_Percent': ['Field may not be null.'], 'Inlet_Width': ['Field may not be null.'], 'Outlet_Height': ['Field may not be null.'], 'Crossing_Structure_Length': ['Field may not be null.'], 'Outlet_Width': ['Field may not be null.'], 'Capacity': ['Required culvert geometry is missing']}",None,None,None,None,None,-1.0,None,None,None,None,None,None,None,None,0.04,0.7,-0.5,None,None,[],False,None
66742,xy4210207573548535,Flow-Aligned,0.868200103,Reduced AOP,true,34,Low/Estimated,"Jastremski, Michael",None,OK,Spans Full Channel & Banks,Bridge,"Jastremski, Michael",1774,2019-01-23 09:40:00.03,2019-01-17 15:46:16.373,2019-01-17 15:46:16.937,2019-01-17,Insignificant barrier,No data,Typical low-flow,42.102075,-73.548535,-73.548579,42.102078,3.6,Headwall and Wingwalls,1827192.044,2339175.387,1827192.044,2339175.387,None,1st bridge on 7a after empire road intersection,false,false,1,"Larson, Lindsay",County Route 7a,0,Paved,None,NY,Bish Bash Brook,None,No,Copake,64662,None,None,None,35,0,Yes,-1,At Stream Grade,6.8,10.891,Box/Bridge with Abutments,48.5,Headwall and Wingwalls,0.89,64.5,None,None,Combination,Middle Hudson,0,0,At Stream Grade,8,12.974,Box/Bridge with Abutments,32.2,0.96,64.5,5,None,-1,No data,None,None,None,Yes,Yes,Columbia,None,None,Wingwall and Headwall,Box,19.659599999999998,2.07264,0.0,-1.0,10.668,None,64.5,8.0,None,40

In [49]:
etl.selectne(hydrated_table, 'comments', [])

Survey_Id,Crossing_Code,Alignment,Aqua_Pass_Score,AOP,Approved,Bankfull_Width,Bankfull_Width_Confidence,Coordinator,Crossing_Comment,Crossing_Condition,Crossing_Span,Crossing_Type,Data_Checked_Coordinator,Database_Entry_By,Date_Data_Checked,Date_First_Entered,Date_Last_Updated,Date_Observed,Evaluation,Flag_Name,Flow_Condition,GIS_Latitude,GIS_Longitude,GPS_X_Coordinate,GPS_Y_Coordinate,GPS_Distance,Inlet_Type,Lccx,Lccy,Lccx_Moved,Lccy_Moved,Local_Id,Location_Description,Maine_Private,No_Crossing,Number_Of_Culverts,Observer,Road,Road_Fill_Height,Road_Type,Scour_Pool,State,Stream_Name,Terrestrial_Passage_Score,Tidal_Site,Town,Naacc_Culvert_Id,Armoring,Barrier_Name,Barrier_Severity,Crossing_Structure_Length,Culvert_Condition_Assess_Id,Dry_Passage,Inlet_Abutment_Height,Inlet_Grade,Inlet_Height,Inlet_Openness,Inlet_Structure_Type,Inlet_Substrate_Water_Width,Inlet_Type1,Inlet_Water_Depth,Inlet_Width,Internal_Structure,Internal_Structure_Comment,Material,NHD_HUC8_Watershed,Outlet_Drop_To_Stream_Bottom,Outlet_Drop_To_Water_Surface,Outlet_Grade,Outlet_Height,Outlet_Openness,Outlet_Structure_Type,Outlet_Substrate_Water_Width,Outlet_Water_Depth,Outlet_Width,Passage_Height,Slope_Confidence,Slope_Percent,Structure_Comment,Structure_Substrate_Matches_Stream,Substrate_Continuous,Substrate_Type,Water_Depth_Matches_Stream,Water_Velocity,County,validation_errors,culv_mat,in_type,in_shape,in_a,in_b,hw,slope,length,out_shape,out_a,out_b,crossing_type,culvert_area_sqm,culvert_depth_m,coefficient_c,coefficient_y,coefficient_slope,slope_rr,head_over_invert,comments,include,culvert_capacity


## Create ArcGIS feature data from the Points

We need something that will work with arcpy tools

In [29]:
from arcpy import CreateFeatureclass_management, FeatureSet, AddFields_management
from arcpy.da import InsertCursor
from arcpy import env

In [30]:
env.overwriteOutput = True
# Create an in_memory feature class to initially contain the points
feature_class = CreateFeatureclass_management("in_memory", "tempfc", "POINT")
AddFields_management(
    feature_class, 
    [
        #[Field Name, Field Type]
        ['uid', 'TEXT', 64],
        ['group_id', 'TEXT', 64],
    ]
)
#fields = list(Point.__dataclass_fields__.keys())
#fields.append("SHAPE@XY")
#fields

<Result 'in_memory\\tempfc'>

In [31]:
# Open an insert cursor
with InsertCursor(feature_class, ['uid', 'group_id', "SHAPE@XY"]) as cursor:
    # Iterate through list of coordinates and add to cursor
    for pt in points:
        row = [pt.uid, pt.group_id, (pt.lng, pt.lat)]
        cursor.insertRow(row)

# Create a FeatureSet object and load in_memory feature class
feature_set = FeatureSet()
feature_set.load(feature_class)
fsd = json.loads(feature_set.JSON)

In [32]:
fsd

{'displayFieldName': '',
 'fieldAliases': {'OID': 'OID', 'uid': '64', 'group_id': '64'},
 'geometryType': 'esriGeometryPoint',
 'spatialReference': {'wkid': None},
 'fields': [{'name': 'OID', 'type': 'esriFieldTypeOID', 'alias': 'OID'},
  {'name': 'uid', 'type': 'esriFieldTypeString', 'alias': '64'},
  {'name': 'group_id', 'type': 'esriFieldTypeString', 'alias': '64'}],
 'features': [{'attributes': {'OID': 1, 'uid': None, 'group_id': '66697'},
   'geometry': {'x': -73.5999755859375, 'y': 42.16351318359375}},
  {'attributes': {'OID': 2, 'uid': '64662', 'group_id': '66742'},
   'geometry': {'x': -73.54852294921875, 'y': 42.10211181640625}},
  {'attributes': {'OID': 3, 'uid': '64663', 'group_id': '66743'},
   'geometry': {'x': -73.5181884765625, 'y': 42.1201171875}},
  {'attributes': {'OID': 4, 'uid': '64664', 'group_id': '66744'},
   'geometry': {'x': -73.5076904296875, 'y': 42.11669921875}},
  {'attributes': {'OID': 5, 'uid': '64672', 'group_id': '66750'},
   'geometry': {'x': -73.62060

## Create Points from ArcGIS feature class

If the NAACC data has been imported into a feature class (that **is not** a shapefile) prior to running the tool, then we take a slightly different approach by using ArcPy to read the feature class, then turn it into a table that is then passed through the NAACC ETL process.

`FeatureSet` can read in any feature class format and gets us an easy way to generate a PETL table.

In [33]:
fs2 = FeatureSet(r"D:\Dropbox (CivicMapper)\Projects\202004-02 Cornell Modeling\3 - Production\tool outputs\c19 tests\c19_snapped_points.shp")
fs2

<FeatureSet object at 0x22bccb6c508[0x22bcd06fe90]>

In [34]:
fs2d = json.loads(fs2.JSON)

In [35]:
fs2d.keys()

dict_keys(['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields', 'features'])

In [36]:
fs2d['features'][0]

{'attributes': {'FID': 0,
  'OBJECTID': 1,
  'pointid': 1,
  'grid_code': 70978,
  'Survey_ID': 73607,
  'NAACC_ID': 70978,
  'Lat': 42.35717,
  'Long': -73.556041,
  'Rd_Name': 'Raup Road',
  'Culv_Mat': 'Metal',
  'In_Type': 'Projecting',
  'In_Shape': 'Round',
  'In_A': 4.9,
  'In_B': 4.8,
  'HW': 2,
  'Slope': 2.8,
  'Length': 26,
  'Out_Shape': 'Round Culvert',
  'Out_A': 5.2,
  'Out_B': 5.4,
  'Crossing_T': 'Culvert',
  'Comments': ' ',
  'Flags': 1,
  'Modeling_n': ' ',
  'BarrierID': '96C19',
  'BarrierID_': 0,
  'BarrierID1': 0},
 'geometry': {'x': 618918.2765349224, 'y': 4690423.416399388}}

Use the content of the `features` object to generate a table; unpack the `attributes` property to turn it into columns. That becomes our starting point for ETL of NAACC data.

In [37]:
t = etl.fromdicts(fs2d['features']).unpackdict('attributes')
t

geometry,BarrierID,BarrierID1,BarrierID_,Comments,Crossing_T,Culv_Mat,FID,Flags,HW,In_A,In_B,In_Shape,In_Type,Lat,Length,Long,Modeling_n,NAACC_ID,OBJECTID,Out_A,Out_B,Out_Shape,Rd_Name,Slope,Survey_ID,grid_code,pointid
"{'x': 618918.2765349224, 'y': 4690423.416399388}",96C19,0,0,,Culvert,Metal,0,1,2,4.9,4.8,Round,Projecting,42.35717,26,-73.556041,,70978,1,5.2,5.4,Round Culvert,Raup Road,2.8,73607,70978,1
"{'x': 610968.2765349215, 'y': 4690193.416399388}",121C19,0,0,headwall on outlet side. large pond downstream,Culvert,Concrete,1,1,5,1.5,1.3,Round,Mitered to Slope,42.356053,42,-73.652552,,71056,2,1.5,1.5,Round Culvert,Mile Hill Road,-1,73685,71056,2
"{'x': 618868.2765349215, 'y': 4690163.416399388}",97C19,0,0,,Culvert,Concrete,2,1,7,6,5.1,Box,Wingwall and Headwall,42.354772,35,-73.556639,,70979,3,5.9,5,Box Culvert,Red Rock Road,6.8,73608,70979,3
"{'x': 618848.2765349206, 'y': 4690093.416399386}",220C19,0,0,,Bridge,Combination,3,1,0,19.9,7.1,Box,Wingwall,42.354158,20.4,-73.556776,,73324,4,20,7.2,Box/Bridge with Abutments,Old road,-1,76071,73324,4
"{'x': 612348.2765349206, 'y': 4690083.416399386}",115C19,0,0,buried upstream under field. fish in scour pool,Culvert,Concrete,4,1,1.5,3,3,Round,Projecting,42.354929,40,-73.635924,,71049,5,3,3,Round Culvert,Church Sreet,3,73678,71049,5
